In [173]:
import boto3
import numpy as np
import json
from pathlib import Path

In [174]:
from scipy.stats import spearmanr

In [175]:
with open(Path(r'../access.json'), 'r') as access_file:
    access = json.load(access_file)
    access_file.close()

In [176]:
client = boto3.client('dynamodb',region_name='us-east-2',aws_access_key_id=access['KEY_IAM_AWS'],aws_secret_access_key=access['SECRET_IAM_AWS'])

In [177]:
def get_ticker(ticker):
    response = client.get_item(TableName='tickers_sp', Key={'ticker': {'S': ticker}})
    return response

In [178]:
def price_process(response):
    price_dict = {}
    for i in response['Item']['price']['L']:
        keys = list(i['M'].keys())[0]
        values = list(list(i['M'].values())[0].values())[0]
            
        price_dict[keys] = values
    return price_dict

In [179]:
def ticker_process(ticker_name: str()) -> dict():
    try:
        ticker = get_ticker(ticker_name)
    except:
        Exception(f'Failed to retrieve data for {ticker_name}')
    
    try:
        ticker_price_clean = price_process(ticker)
    except:
        Exception(f'Failed to process data for {ticker_name}')
    
    return ticker_price_clean

In [180]:
def reverse_order(ticker_dict: dict()) -> dict():
    """
    reverses dict ordering (from descending to ascending)
    """
    return dict(reversed(list(ticker_dict.items())))

In [172]:
def full_process(ticker_names: list()):
    name_1 = ticker_names[0]
    name_2 = ticker_names[1]

    ticker_1 = ticker_process(name_1)
    ticker_2 = ticker_process(name_2)

    mutual_keys = []

    for key in ticker_1:
        if key in ticker_2:
            mutual_keys.append(key)
    
    ticker_1_list = []
    ticker_2_list = []
    for i in mutual_keys:
        t1 = ticker_1[i]
        t2 = ticker_2[i]
        ticker_1_list.append(float(t1))
        ticker_2_list.append(float(t2))    
    
    ticker_1_asc = list(reversed(ticker_1_list))
    ticker_2_asc = list(reversed(ticker_2_list))

    t1_diff = np.diff(ticker_1_asc) / np.abs(ticker_1_asc[:-1]) * 100
    t2_diff = np.diff(ticker_2_asc) / np.abs(ticker_2_asc[:-1]) * 100

    cor_val = spearmanr(t1_diff, t2_diff, nan_policy='omit')

    return cor_val

In [86]:
ticker_unclean_2 = get_ticker('AMP')

In [88]:
ticker_2 = price_process(ticker_unclean_2)

In [101]:
ticker_1 = price_dict

In [155]:
mutual_keys = []
for key in ticker_1:
    if key in ticker_2:
        mutual_keys.append(key)

In [157]:
ticker_1_list = []
ticker_2_list = []
for i in mutual_keys:
    t1 = ticker_1[i]
    t2 = ticker_2[i]
    ticker_1_list.append(float(t1))
    ticker_2_list.append(float(t2))

In [161]:
ticker_1_asc = list(reversed(ticker_1_list))
ticker_2_asc = list(reversed(ticker_2_list))

In [166]:
t1_diff = np.diff(ticker_1_asc) / np.abs(ticker_1_asc[:-1]) * 100
t2_diff = np.diff(ticker_2_asc) / np.abs(ticker_2_asc[:-1]) * 100

In [170]:
cor_val = spearmanr(t1_diff, t2_diff, nan_policy='omit')

In [171]:
cor_val

SpearmanrResult(correlation=0.46343685892848613, pvalue=1.3342950574796146e-08)